In [2]:
from model import *
import pytchat


C:\Users\avika\OneDrive\Documents\UAL\interactive_dance_thesis


In [2]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="michellejieli/emotion_text_classifier")
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("michellejieli/emotion_text_classifier")
model = AutoModelForSequenceClassification.from_pretrained("michellejieli/emotion_text_classifier")

In [3]:
# Preparing MEED data for model
MEED= prep_MEED_data()
train_data,val_data,frame_dim,max_x,min_x,max_y,min_y,max_dx,min_dx,max_dy,min_dy = MEED

100%|██████████| 4102/4102 [00:46<00:00, 87.81it/s] 


Validating interpolation...
No errors found!


4102it [00:01, 2400.03it/s]
4102it [00:01, 2274.56it/s]


Creating keypoint frames...


100%|██████████| 4102/4102 [00:03<00:00, 1061.08it/s]


...

Validating length of dkp_frames
Validating length of kp_frames
Adding deltas to frames...


4102it [00:02, 1645.57it/s]


Validating length of data after delta
Adding emotions to frames...


100%|██████████| 4102/4102 [00:00<00:00, 29193.49it/s]


Validating length of data after emotions
frame_dim: 107


In [4]:
emotion_labels = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']

In [5]:

def visualise_skeleton(all_frames, max_x, max_y, max_frames=500, save=False, save_path=None, prefix=None, train_seed=None , delta=False, destroy = True):
    """Input all frames dim 50xn n being the number of frames 50= 25 keypoints x and y coordinates"""

    
    # visualise to check if the data is correct
    # BODY_25 Keypoints
    keypointsMapping = ['Nose', 'Neck', 'R-Sho', 'R-Elb', 'R-Wr', 'L-Sho', 
                        'L-Elb', 'L-Wr', 'MidHip', 'R-Hip', 'R-Knee', 'R-Ank', 
                        'L-Hip', 'L-Knee', 'L-Ank', 'R-Eye', 'L-Eye', 'R-Ear', 
                        'L-Ear', 'L-BigToe', 'L-SmallToe', 'L-Heel', 'R-BigToe', 
                        'R-SmallToe', 'R-Heel']


    limb_connections = [
        ("Nose", "Neck"),
        ("Neck", "R-Sho"),
        ("R-Sho", "R-Elb"),
        ("R-Elb", "R-Wr"),
        ("Neck", "L-Sho"),
        ("L-Sho", "L-Elb"),
        ("L-Elb", "L-Wr"),
        ("Neck", "MidHip"),
        ("MidHip", "R-Hip"),
        ("R-Hip", "R-Knee"),
        ("R-Knee", "R-Ank"),
        ("MidHip", "L-Hip"),
        ("L-Hip", "L-Knee"),
        ("L-Knee", "L-Ank"),
        ("Nose", "R-Eye"),
        ("R-Eye", "R-Ear"),
        ("Nose", "L-Eye"),
        ("L-Eye", "L-Ear"),
        ("L-Ank", "L-BigToe"),
        ("L-Ank", "L-SmallToe"),
        ("L-Ank", "L-Heel"),
        ("R-Ank", "R-BigToe"),
        ("R-Ank", "R-SmallToe"),
        ("R-Ank", "R-Heel")
    ]
    
     # Define a mapping from emotion vectors to emotion labels
    # Define emotion labels
    emotion_labels = ['Anger', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sad', 'Surprise']
    
    # Initialize a blank canvas (image)
    canvas_size = (int(max_y)+50, int(max_x)+50, 3)  
    canvas = np.zeros(canvas_size, dtype=np.uint8)
    
    
    if save:
        # Determine the save path
        if save_path is None:
            save_path = f"D:\\Interactive Dance Thesis Tests\\TransformerResults\\{train_seed}"

        # Ensure directory exists
        if not os.path.exists(save_path):
            print(f"Creating directory {save_path}")
            os.makedirs(save_path)

        # Determine a unique filename
        existing_files = os.listdir(save_path)
        file_num = 1
        while f"{prefix or ''}{file_num}.mp4" in existing_files:
            file_num += 1
        out_path = os.path.join(save_path, f"{prefix or ''}{file_num}.mp4")

        # Create the video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(out_path, fourcc, 10.0, (canvas_size[1], canvas_size[0]))
    
    previous_frame_data = None
    
    # Iterate over all frames; the first frame uses absolute keypoints, the rest use relative keypoints (deltas)
    for frame_data in tqdm(all_frames[:max_frames], desc="Visualizing frames"):
        
        # If previous_frame_data is None, this is the first frame and we use absolute positions.
        # Otherwise, add the delta to the previous frame's keypoints to get the new keypoints
        if delta ==True:
            if previous_frame_data is not None:
                frame_data[:50] = [prev + delta for prev, delta in zip(previous_frame_data[:50], frame_data[50:100])]
        
            # Update previous_frame_data
            previous_frame_data = copy.deepcopy(frame_data)
        
        canvas_copy = canvas.copy()
        
        # Extract x, y coordinates and emotion vector
        x_coords = frame_data[0:50:2] 
        y_coords = frame_data[1:50:2]
        emotion_vector = tuple(frame_data[100:107])
        
        xy_coords = list(zip(x_coords, y_coords))
        sane = sanity_check(xy_coords)
        # Plot keypoints on the canvas
        for i, (x, y) in enumerate(xy_coords):
            if sane[i] == False:
                continue
            x_val = x.item() if torch.is_tensor(x) else x
            y_val = y.item() if torch.is_tensor(y) else y
            cv2.circle(canvas_copy, (int(x_val), int(y_val)), 3, (0, 0, 255), -1)  
            cv2.putText(canvas_copy, keypointsMapping[i], (int(x_val), int(y_val)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

        # Draw connections (limbs) on the canvas
        for limb in limb_connections:
            start_idx = keypointsMapping.index(limb[0])
            end_idx = keypointsMapping.index(limb[1])
            
            start_point = (int(x_coords[start_idx]), int(y_coords[start_idx]))
            end_point = (int(x_coords[end_idx]), int(y_coords[end_idx]))

            if start_point == (0,0) or end_point == (0,0) or not sane[start_idx] or not sane[end_idx]:
                continue
            cv2.line(canvas_copy, start_point, end_point, (0, 255, 0), 2)  
        
        # Display the emotion percentages and labels on the top right of the frame
        
        emotion_percentages = [f"{int(e * 100)}% {emotion_labels[i]}" for i, e in enumerate(emotion_vector) if round(e * 100) > 1]
        # print(emotion_percentages)
        y0, dy = 30, 15  # Starting y position and line gap
        for i, line in enumerate(emotion_percentages):
            y = y0 + i * dy
            cv2.putText(canvas_copy, line, (canvas_size[1] - 120, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)


        # Display the canvas with keypoints and connections
        cv2.imshow("Keypoints Visualization", canvas_copy)

        # If saving, write the frame to the video
        if save:
            out.write(canvas_copy)

        # Wait for 100ms and check for "esc" key press to exit
        key = cv2.waitKey(100)
        if key == 27:  
            break

    # Release the video writer, if used
    if save:
        out.release()

    if destroy:
        # Close the visualization window
        cv2.destroyAllWindows()


In [6]:


# Initialize a dictionary to keep track of cumulative scores and counts for each emotion
emotion_data = {emotion: {"score": 0.0, "count": 0} for emotion in emotion_labels}


# List to store average scores indexed by emotions
average_scores = [0.0 for _ in emotion_labels]
length =0

BLOCK_SIZE = 16 # what is the maximum context length for predictions? 

# create model
CHECKPOINT_PATH = "checkpoints/proto6_checkpoint.pth"

m = MotionModel(input_dim=frame_dim, output_dim=frame_dim, blocksize=BLOCK_SIZE, hidden_dim=512, n_layers=8, dropout=DROPOUT)
m = m.to(device)
optimizer = torch.optim.Adam(m.parameters(), lr=LEARNING_RATE, weight_decay=L2_REG)  # weight_decay=1e-5 L2 regularization
m, optimizer,scheduler, epoch, loss, train_seed = load_checkpoint(m, optimizer, CHECKPOINT_PATH)
print(f"training seed: {train_seed}")


Loading checkpoint...
Checkpoint loaded from checkpoints/proto6_checkpoint.pth
training seed: 33694


In [7]:

chosen_emotion = "Fear"
specific_emotion_video = get_video_by_emotion(train_data, chosen_emotion)
specific_emotion_video.shape

torch.Size([1, 97, 107])

In [8]:
FRAMES_GENERATE = 150

chosen_emotion = "Fear"
# seems like 150 frames is good before it get's a bit weird

specific_emotion_video = get_video_by_emotion(train_data, chosen_emotion)

# initial generation always neutral

generated = m.generate(specific_emotion_video, FRAMES_GENERATE)
unnorm_out = unnormalise_list_2D(generated, max_x, min_x, max_y, min_y,max_x, min_x, max_y, min_y)

# visualise
visualise_skeleton(unnorm_out[0], max_x, max_y, max_frames=FRAMES_GENERATE,save = True,save_path=None,prefix=f'{chosen_emotion}_{epoch}_coord',train_seed=train_seed,delta=False)


# have to normalise the generated data before putting it into the model

Visualizing frames: 100%|██████████| 150/150 [00:16<00:00,  9.22it/s]


In [11]:
FRAMES_GENERATE = 500


noise = torch.randn(1, 5, 107).to(device)
print(noise.shape)  # Should print: torch.Size([1, 97, 107])


# initial generation always neutral

generated = m.generate(noise, FRAMES_GENERATE)
unnorm_out = unnormalise_list_2D(generated, max_x, min_x, max_y, min_y,max_x, min_x, max_y, min_y)

# visualise
visualise_skeleton(unnorm_out[0], max_x, max_y, max_frames=FRAMES_GENERATE,save = True,save_path=None,prefix=f'noise__{epoch}_coord',train_seed=train_seed,delta=False)


# have to normalise the generated data before putting it into the model

torch.Size([1, 5, 107])


Visualizing frames: 100%|██████████| 500/500 [00:54<00:00,  9.22it/s]


In [10]:
def normalise_generated(unnorm_out, max_x, min_x, max_y, min_y, max_dx, min_dx, max_dy, min_dy):
    norm_out = []
    
    for frame in unnorm_out:
        norm_frame = []
        
        # Normalize the first 50 values (absolute x and y coordinates)
        for i in range(0, 50, 2):
            unnormalized_x = frame[i]
            unnormalized_y = frame[i+1]
            
            norm_x = 2 * (unnormalized_x - min_x) / (max_x - min_x) - 1
            norm_y = 2 * (unnormalized_y - min_y) / (max_y - min_y) - 1
            
            norm_frame.extend([norm_x, norm_y])
        
        # Normalize the second 50 values (x and y deltas)
        for i in range(50, 100, 2):
            unnormalized_dx = frame[i]
            unnormalized_dy = frame[i+1]
            
            norm_dx = 2 * (unnormalized_dx - min_dx) / (max_dx - min_dx) - 1
            norm_dy = 2 * (unnormalized_dy - min_dy) / (max_dy - min_dy) - 1
            
            norm_frame.extend([norm_dx, norm_dy])
        
        # Append the emotion encoding without normalizing
        norm_frame.extend(frame[-7:])
        norm_out.append(norm_frame)
        
    return norm_out


In [11]:
import threading
import queue
import time
import keyboard
import queue


In [12]:
import glob

files = glob.glob("D:/Interactive Dance Thesis Tests/TransformerResults/*/*500000*")
print(files)

['D:/Interactive Dance Thesis Tests/TransformerResults\\26277\\adam_500000steps_optimised_6layer_dropout_lerp1.mp4', 'D:/Interactive Dance Thesis Tests/TransformerResults\\26277\\adam_500000steps_optimised_6layer_dropout_lerp2.mp4', 'D:/Interactive Dance Thesis Tests/TransformerResults\\26277\\adam_500000steps_optimised_6layer_dropout_lerp3.mp4', 'D:/Interactive Dance Thesis Tests/TransformerResults\\26277\\adam_500000steps_optimised_6layer_dropout_lerp4.mp4', 'D:/Interactive Dance Thesis Tests/TransformerResults\\33694\\adam_500000_coord1.mp4', 'D:/Interactive Dance Thesis Tests/TransformerResults\\33694\\adam_500000_coord2.mp4', 'D:/Interactive Dance Thesis Tests/TransformerResults\\33694\\adam_500000_coord3.mp4', 'D:/Interactive Dance Thesis Tests/TransformerResults\\33694\\adam_500000_coord4.mp4', 'D:/Interactive Dance Thesis Tests/TransformerResults\\33694\\adam_500000_coord5.mp4', 'D:/Interactive Dance Thesis Tests/TransformerResults\\33694\\adam_500000_coord6.mp4', 'D:/Interacti

In [13]:

# Initial setup
shared_data = {
    'average_scores': [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
}

chat = pytchat.create(video_id="gCNeDWCI0vo")
FRAMES_GENERATE = 150
terminate_threads = False

# This queue will hold the batches ready for visualization
viz_queue = queue.Queue()


def process_chat_message(c):
    """Process a chat message and update emotion scores."""
    print(f"{c.datetime} [{c.author.name}]- {c.message}")
    result = pipe(c.message)  # Assuming pipe() returns emotion prediction
    print(result)

    detected_emotion = result[0]['label']

    # Reset the counter for the detected emotion and boost its score
    emotion_data[detected_emotion]["count"] = 0
    score = result[0]['score']
    emotion_data[detected_emotion]["score"] = min(1, emotion_data[detected_emotion]["score"] + score)

    # Decay scores for other emotions and increase their counters
    for emotion, data in emotion_data.items():
        if emotion != detected_emotion:
            data["count"] += 1
            if data["count"] >= 5:
                data["score"] = 0
            else:
                data["score"] *= 0.5  # or any other decay factor you prefer

    # Update average scores
    for i, emotion in enumerate(emotion_labels):
        shared_data['average_scores'][i] = emotion_data[emotion]["score"]

    print("Average scores:", shared_data['average_scores'])


INPUT_LENGTH = 1

# Batch generation function
def generate_new_batch():
    """Generate a new batch based on the current average scores."""
    last_frames = unnorm_out[0][-INPUT_LENGTH:]
    norm_last_frames = normalise_generated(last_frames, max_x, min_x, max_y, min_y, max_x, min_x, max_y, min_y)
    new_batch = [frame[:-7] + shared_data['average_scores'] for frame in norm_last_frames]
    new_input = torch.tensor([new_batch]).to(device).float()

    # Generate the new frames
    generated = m.generate(new_input, FRAMES_GENERATE)
    return unnormalise_list_2D(generated, max_x, min_x, max_y, min_y, max_x, min_x, max_y, min_y)

# Periodic batch generation function - secs
def generate_batches_periodically(period=2):
    """Periodically generate batches and put them in the visualization queue."""
    while True:
        time.sleep(period)
        unnorm_out = generate_new_batch()
        viz_queue.put(unnorm_out)
        
def visualise(unnorm_out):
    # visualize
    visualise_skeleton(unnorm_out[0], max_x, max_y, max_frames=FRAMES_GENERATE,save = False,save_path=None,prefix=f'adam_{EPOCHS}_coord_switchhappytodisgust',train_seed=train_seed,delta=False,destroy=False)

# Visualization thread
def visualise_batches():
    while True:
        # Wait for a new batch to be available in the queue
        unnorm_out = viz_queue.get()
        
        # visualise the batch
        visualise(unnorm_out)

# Start the threads
threading.Thread(target=visualise_batches, daemon=True).start()
threading.Thread(target=generate_batches_periodically, args=(10,), daemon=True).start()

# Process chat messages
length = 0
while chat.is_alive():
    if keyboard.is_pressed('esc'):  # Check if ESC key is pressed
        terminate_threads = True
        break  # Exit the main loop
    for c in chat.get().sync_items():
        process_chat_message(c)
        length += 1
        
cv2.destroyAllWindows()


2023-10-25 17:24:29 [Wayz ezY]- They insane ese:hugging_face::dove:
[{'label': 'disgust', 'score': 0.631585955619812}]
Average scores: [0.0, 0.631585955619812, 0.0, 0.0, 0.0, 0.0, 0.0]
2023-10-25 17:24:29 [Open AI Channel / Sites Reviews]- Make Gaza a golf course 
[{'label': 'neutral', 'score': 0.9495328664779663}]
Average scores: [0.0, 0.315792977809906, 0.0, 0.0, 0.9495328664779663, 0.0, 0.0]
2023-10-25 17:24:36 [Bitsmart]- paperchase, Allah is not idol. I disagree, quran mention about Idolatry and it's clear. moreover you can blame your Idolatry
[{'label': 'anger', 'score': 0.4567664563655853}]
Average scores: [0.4567664563655853, 0.157896488904953, 0.0, 0.0, 0.47476643323898315, 0.0, 0.0]


Visualizing frames:   3%|▎         | 5/150 [00:00<00:16,  9.04it/s]

2023-10-25 17:24:42 [Michael Ryan]- If you know Noah’s sent his kids out into the world after the flood Sodom Gamorrah we all belong to one of the 12 tribes we all decend from Noah and kids we are one peoples humans some act like beasts
[{'label': 'neutral', 'score': 0.8912690877914429}]
Average scores: [0.22838322818279266, 0.0789482444524765, 0.0, 0.0, 1, 0.0, 0.0]


Visualizing frames:  30%|███       | 45/150 [00:04<00:11,  9.20it/s]

2023-10-25 17:24:46 [Jack Field]- Gaza needs fuel? Ask their leadership, they have tons of stolen fuel and medicine
[{'label': 'anger', 'score': 0.5463253855705261}]
Average scores: [0.7747086137533188, 0.03947412222623825, 0, 0, 0.5, 0, 0]


...

Visualizing frames:  12%|█▏        | 18/150 [00:01<00:14,  9.21it/s]

2023-10-25 17:28:48 [Rain Tarrant]- Poor children :(
[{'label': 'sadness', 'score': 0.9613035917282104}]
Average scores: [0, 0, 0, 0, 0.5, 0.9613035917282104, 0.18846048787236214]


Visualizing frames:  25%|██▌       | 38/150 [00:04<00:12,  9.24it/s]

2023-10-25 17:28:49 [Shakeel Ahmed]- Bravery is not killing innocent Children and women. Israel has most coward army on this planet killing innocent women and children. If the world wants peace this terrorist state must be removed
[{'label': 'anger', 'score': 0.42462512850761414}]
Average scores: [0.42462512850761414, 0, 0, 0, 0.25, 0.4806517958641052, 0.09423024393618107]


Visualizing frames:  31%|███       | 46/150 [00:04<00:11,  9.28it/s]

In [45]:
from pytube import YouTube

yt = YouTube('https://www.youtube.com/watch?v=OlmMP2Z83Pg')
yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution') .desc().first().download()

'C:\\Users\\avika\\OneDrive\\Documents\\UAL\\interactive_dance_thesis\\Sean Lew  Low - SZA  Dance Choreography by Alexander Chung.mp4'

In [52]:
import os
import pandas as pd
import subprocess
from tqdm import tqdm

def download_videos(csv_path, output_dir):
    # Create the output directory if it does not exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Load the CSV file
    df = pd.read_csv(csv_path, header=None, names=["video_url"])

    # Iterate over the video URLs in the CSV file
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Downloading videos"):
        video_url = row['video_url']  # Assuming the column name is 'video_url'
        video_name = os.path.basename(video_url)
        output_path = os.path.join(output_dir, video_name)

        # Download the video using wget
        subprocess.run(['wget', '-O', output_path, video_url, '--show-progress'])

if __name__ == "__main__":
    csv_path = "G:/Downloads/refined_10M_all_video_url.csv"
    output_dir = "G:/UAL_Thesis/affective_computing_datasets/AIST"
    download_videos(csv_path, output_dir)


In [62]:
import glob
files = glob.glob("G:\\UAL_Thesis\\affective_computing_datasets\\DanceDBrenders\\DanceDB\\*\\*_keypoints.txt")
files

['G:\\UAL_Thesis\\affective_computing_datasets\\DanceDBrenders\\DanceDB\\20120731_StefanosTheodorou\\Stefanos_1os_antrikos_karsilamas_C3D_body_keypoints.txt',
 'G:\\UAL_Thesis\\affective_computing_datasets\\DanceDBrenders\\DanceDB\\20120731_StefanosTheodorou\\Stefanos_2os_antrikos_karsilamas_C3D_body_keypoints.txt',
 'G:\\UAL_Thesis\\affective_computing_datasets\\DanceDBrenders\\DanceDB\\20120731_StefanosTheodorou\\Stefanos_3os_antrikos_karsilamas_C3D_body_keypoints.txt',
 'G:\\UAL_Thesis\\affective_computing_datasets\\DanceDBrenders\\DanceDB\\20120731_StefanosTheodorou\\Stefanos_Zeibekiko_CY_C3D_body_keypoints.txt',
 'G:\\UAL_Thesis\\affective_computing_datasets\\DanceDBrenders\\DanceDB\\20120805_VasoAristeidou\\Vaso_Aristeidou_Zeimpekiko_v1_body_keypoints.txt',
 'G:\\UAL_Thesis\\affective_computing_datasets\\DanceDBrenders\\DanceDB\\20120805_VasoAristeidou\\Vasso_Aristidou_Hasapiko_body_keypoints.txt',
 'G:\\UAL_Thesis\\affective_computing_datasets\\DanceDBrenders\\DanceDB\\20120807_

In [ ]:
emotions_set = {
    'Happy', 'Miserable', 'Relaxed', 'Sad', 'Satisfied', 'Tired', 
    'Excited', 'Afraid', 'Angry', 'Annoyed', 'Bored', 'Pleased',
    'Neutral','Nervous','Mix','Curiosity'
}


emotion_labels = ['Anger', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sad', 'Surprise']

In [15]:
import glob

emotions_set = {
    'Happy', 'Miserable', 'Relaxed', 'Sad', 'Satisfied', 'Tired', 
    'Excited', 'Afraid', 'Angry', 'Annoyed', 'Bored', 'Pleased',
    'Neutral', 'Nervous', 'Mix', 'Curiosity'
}

# Define the path pattern to load the files
path_pattern = "G:\\UAL_Thesis\\affective_computing_datasets\\DanceDBrenders\\DanceDB\\*\\*_keypoints.txt"

# Get the list of all files that match the pattern
files = glob.glob(path_pattern)

# Filter the files based on the keywords in emotions_set
filtered_files = [file for file in files if any(emotion.lower() in file.lower() for emotion in emotions_set)]

print("Filtered files:")
for file in filtered_files:
    print(file)


Filtered files:
G:\UAL_Thesis\affective_computing_datasets\DanceDBrenders\DanceDB\20130216_AnnaCharalambous\Anna_Angry_C3D_body_keypoints.txt
G:\UAL_Thesis\affective_computing_datasets\DanceDBrenders\DanceDB\20130216_AnnaCharalambous\Anna_Curiosity_C3D_body_keypoints.txt
G:\UAL_Thesis\affective_computing_datasets\DanceDBrenders\DanceDB\20130216_AnnaCharalambous\Anna_Happy_C3D_body_keypoints.txt
G:\UAL_Thesis\affective_computing_datasets\DanceDBrenders\DanceDB\20130216_AnnaCharalambous\Anna_Nervous_C3D_body_keypoints.txt
G:\UAL_Thesis\affective_computing_datasets\DanceDBrenders\DanceDB\20130216_AnnaCharalambous\Anna_Sad_C3D_body_keypoints.txt
G:\UAL_Thesis\affective_computing_datasets\DanceDBrenders\DanceDB\20131001_OliviaKyriakides\Olivia_Annoyed_C3D_body_keypoints.txt
G:\UAL_Thesis\affective_computing_datasets\DanceDBrenders\DanceDB\20131001_OliviaKyriakides\Olivia_Bored_C3D_body_keypoints.txt
G:\UAL_Thesis\affective_computing_datasets\DanceDBrenders\DanceDB\20131001_OliviaKyriakides\

In [13]:
import glob

emotions_set = {
    'Happy', 'Miserable', 'Relaxed', 'Sad', 'Satisfied', 'Tired', 
    'Excited', 'Afraid', 'Angry', 'Annoyed', 'Bored', 'Pleased',
    'Neutral', 'Nervous', 'Mix', 'Curiosity'
}

# Define the path pattern to load the files
path_pattern = "G:\\UAL_Thesis\\affective_computing_datasets\\DanceDBrenders\\DanceDB\\*\\*_keypoints.txt"

# Get the list of all files that match the pattern
files = glob.glob(path_pattern)

# Filter the files based on the keywords in emotions_set and return the matched emotion
filtered_files_and_emotions = [(file, emotion) for file in files for emotion in emotions_set if emotion.lower() in file.lower()]

print("Filtered files and matched emotions:")
for file, emotion in filtered_files_and_emotions:
    print(f"{file}: {emotion}")


Filtered files and matched emotions:
G:\UAL_Thesis\affective_computing_datasets\DanceDBrenders\DanceDB\20130216_AnnaCharalambous\Anna_Angry_C3D_body_keypoints.txt: Angry
G:\UAL_Thesis\affective_computing_datasets\DanceDBrenders\DanceDB\20130216_AnnaCharalambous\Anna_Curiosity_C3D_body_keypoints.txt: Curiosity
G:\UAL_Thesis\affective_computing_datasets\DanceDBrenders\DanceDB\20130216_AnnaCharalambous\Anna_Happy_C3D_body_keypoints.txt: Happy
G:\UAL_Thesis\affective_computing_datasets\DanceDBrenders\DanceDB\20130216_AnnaCharalambous\Anna_Nervous_C3D_body_keypoints.txt: Nervous
G:\UAL_Thesis\affective_computing_datasets\DanceDBrenders\DanceDB\20130216_AnnaCharalambous\Anna_Sad_C3D_body_keypoints.txt: Sad
G:\UAL_Thesis\affective_computing_datasets\DanceDBrenders\DanceDB\20131001_OliviaKyriakides\Olivia_Annoyed_C3D_body_keypoints.txt: Annoyed
G:\UAL_Thesis\affective_computing_datasets\DanceDBrenders\DanceDB\20131001_OliviaKyriakides\Olivia_Bored_C3D_body_keypoints.txt: Bored
G:\UAL_Thesis\af

In [16]:
file_path = filtered_files[0]

with open(file_path, 'r') as file:
    content = file.read()
content

'{"Nose": 0, "Neck": 1, "R-Sho": 2, "R-Elb": 3, "R-Wr": 4, "L-Sho": 5, "L-Elb": 6, "L-Wr": 7, "MidHip": 8, "R-Hip": 9, "R-Knee": 10, "R-Ank": 11, "L-Hip": 12, "L-Knee": 13, "L-Ank": 14, "R-Eye": 15, "L-Eye": 16, "R-Ear": 17, "L-Ear": 18, "L-BigToe": 19, "L-SmallToe": 20, "L-Heel": 21, "R-BigToe": 22, "R-SmallToe": 23, "R-Heel": 24, "0": [[377, 311], [377, 378], [311, 378], [222, 378], [133, 400], [422, 378], [511, 400], [622, 378], [377, 578], [333, 578], [333, 690], [333, 801], [400, 578], [400, 712], [422, 823], [377, 289], [400, 289], [355, 311], [400, 311], [422, 845], [444, 845], [422, 823], [333, 845], [311, 845], [333, 823]], "1": [[377, 311], [377, 378], [311, 378], [222, 378], [133, 400], [422, 378], [511, 400], [600, 378], [377, 578], [333, 578], [333, 690], [333, 801], [400, 578], [400, 712], [422, 823], [377, 289], [400, 289], [355, 311], [400, 311], [422, 845], [444, 845], [422, 823], [333, 845], [311, 845], [333, 823]], "2": [[377, 311], [377, 378], [311, 378], [222, 378]

In [21]:
import json

content_dict = json.load(content)


AttributeError: 'str' object has no attribute 'read'

In [12]:

len(content_dict)-25

942

In [14]:
content_dict['941']

[[333, 244],
 [377, 311],
 [422, 333],
 [422, 422],
 [422, 512],
 [355, 311],
 [333, 422],
 [311, 512],
 [377, 512],
 [400, 512],
 [377, 645],
 [355, 756],
 [355, 512],
 [355, 645],
 [444, 779],
 None,
 [333, 244],
 None,
 [355, 244],
 [422, 823],
 [444, 801],
 [466, 779],
 [311, 756],
 [311, 734],
 [377, 756]]

In [20]:

from tqdm import tqdm
flattened_dict={}
for i in tqdm(range(len(content_dict)-25)):
    nested_list = content_dict[str(i)]
     
    flattened_dict[i] = [coordinate for sublist in nested_list for coordinate in (sublist if sublist is not None else [0, 0])]
flattened_dict

100%|██████████| 942/942 [00:00<00:00, 312928.43it/s]


{0: [377,
  311,
  377,
  378,
  311,
  378,
  222,
  378,
  133,
  400,
  422,
  378,
  511,
  400,
  622,
  378,
  377,
  578,
  333,
  578,
  333,
  690,
  333,
  801,
  400,
  578,
  400,
  712,
  422,
  823,
  377,
  289,
  400,
  289,
  355,
  311,
  400,
  311,
  422,
  845,
  444,
  845,
  422,
  823,
  333,
  845,
  311,
  845,
  333,
  823],
 1: [377,
  311,
  377,
  378,
  311,
  378,
  222,
  378,
  133,
  400,
  422,
  378,
  511,
  400,
  600,
  378,
  377,
  578,
  333,
  578,
  333,
  690,
  333,
  801,
  400,
  578,
  400,
  712,
  422,
  823,
  377,
  289,
  400,
  289,
  355,
  311,
  400,
  311,
  422,
  845,
  444,
  845,
  422,
  823,
  333,
  845,
  311,
  845,
  333,
  823],
 2: [377,
  311,
  377,
  378,
  311,
  378,
  222,
  378,
  133,
  400,
  422,
  378,
  511,
  400,
  600,
  378,
  377,
  578,
  333,
  578,
  333,
  690,
  333,
  801,
  400,
  578,
  400,
  712,
  422,
  823,
  377,
  289,
  400,
  289,
  355,
  311,
  400,
  311,
  422,
  845,
  444,
  

In [19]:

from tqdm import tqdm
flattened_dict={}
x=[]
y=[]
for i in tqdm(range(len(content_dict)-25)):
    nested_list = content_dict[str(i)]
    x.extend([coordinate[0] if coordinate is not None else 0 for coordinate in nested_list])
    y.extend([coordinate[1] if coordinate is not None else 0 for coordinate in nested_list])

    


100%|██████████| 942/942 [00:00<00:00, 313873.08it/s]


[377,
 377,
 311,
 222,
 133,
 422,
 511,
 622,
 377,
 333,
 333,
 333,
 400,
 400,
 422,
 377,
 400,
 355,
 400,
 422,
 444,
 422,
 333,
 311,
 333,
 377,
 377,
 311,
 222,
 133,
 422,
 511,
 600,
 377,
 333,
 333,
 333,
 400,
 400,
 422,
 377,
 400,
 355,
 400,
 422,
 444,
 422,
 333,
 311,
 333,
 377,
 377,
 311,
 222,
 133,
 422,
 511,
 600,
 377,
 333,
 333,
 333,
 400,
 400,
 422,
 377,
 400,
 355,
 400,
 422,
 444,
 422,
 333,
 311,
 333,
 377,
 377,
 311,
 222,
 133,
 422,
 511,
 600,
 377,
 333,
 333,
 333,
 400,
 400,
 422,
 377,
 400,
 355,
 400,
 422,
 444,
 422,
 333,
 311,
 333,
 377,
 377,
 311,
 222,
 133,
 422,
 511,
 600,
 377,
 333,
 333,
 333,
 400,
 400,
 422,
 377,
 400,
 355,
 400,
 422,
 444,
 422,
 333,
 311,
 333,
 377,
 377,
 311,
 222,
 133,
 422,
 511,
 600,
 377,
 333,
 333,
 311,
 400,
 400,
 422,
 377,
 400,
 355,
 400,
 422,
 444,
 400,
 311,
 311,
 333,
 377,
 377,
 311,
 222,
 111,
 422,
 511,
 622,
 377,
 333,
 333,
 311,
 400,
 400,
 422,
 377,
 400

In [1]:
from data import *

prep_data(dataset = "DanceDB")

  0%|          | 0/109 [00:00<?, ?it/s]


KeyError: 'x'

In [ ]:
from model import *


In [6]:


def danceDB_emotions():
    # co pilot is really good at guessing emotions combinations
    return {
        'Happy': {'Happiness': 1.0},
        'Miserable': {'Sad': 1.0},
        'Relaxed': {'Neutral': 0.9, 'Happiness': 0.1},
        'Sad': {'Sad': 1.0},
        'Satisfied': {'Happiness': 1.0},
        'Tired': {'Neutral': 1.0},
        'Excited': {'Happiness': 0.9, 'Surprise': 0.1},
        'Afraid': {'Fear': 1.0},
        'Angry': {'Anger': 1.0},
        'Annoyed': {'Anger': 0.5, 'Disgust': 0.5},
        'Bored': {'Neutral': 0.8, 'Sad': 0.1, 'Anger': 0.1},
        'Pleased': {'Happiness': 1.0},
        'Neutral': {'Neutral': 1.0},
        'Nervous': {'Fear': 0.5, 'Surprise': 0.5},
        'Mix': {'Anger': 0.143, 'Disgust': 0.143, 'Fear': 0.143, 'Happiness': 0.143, 'Neutral': 0.143, 'Sad': 0.143, 'Surprise': 0.143},
        'Curiosity': {'Surprise': 0.3,'Happiness': 0.3,'Neutral': 0.3,'Fear': 0.1}
    }

def encode_danceDB_emotion(emotion):
    
    emotion_labels = ['Anger', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sad', 'Surprise']
    
    emotion_mapping = danceDB_emotions()
    if emotion not in emotion_mapping:
        return "Emotion not found in mapping"
    
    encoding = [0.0] * len(emotion_labels)
    for emotion_label, percentage in emotion_mapping[emotion].items():
        if emotion_label in emotion_labels:
            index = emotion_labels.index(emotion_label)
            encoding[index] = percentage
    
    return encoding

# Test the function with an example
example_emotion = 'Annoyed'
encoded_emotion = encode_danceDB_emotion(example_emotion)
print(f"Encoding for '{example_emotion}': {encoded_emotion}")


Encoding for 'Annoyed': [0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0]


In [10]:
set(danceDB_emotions().keys())

{'Afraid',
 'Angry',
 'Annoyed',
 'Bored',
 'Curiosity',
 'Excited',
 'Happy',
 'Miserable',
 'Mix',
 'Nervous',
 'Neutral',
 'Pleased',
 'Relaxed',
 'Sad',
 'Satisfied',
 'Tired'}